In [1]:
import numpy as np
import pickle

np.random.seed(111)

In [2]:
# Input parameters
N = 50
#restrict_digits = None
restrict_digits = [ 1, 7]
Nl = 10 if restrict_digits is None else len(restrict_digits)
Ns = 1 # Number of samples per graph
k = 8 # Number of nearest neighbors in graph
out_fname = 'MNIST_' + str(N) + '.pkl'

In [3]:
# Load the data from the MNIST pickle
filehandler = open('mnist_data.pkl', 'rb')
train_image, train_label, test_image, test_label = pickle.load(filehandler)

In [4]:
# Only retain required digits
def restrict_data(img, label, restrict_digits):
    label_int = [ np.where(r==1)[0][0] for r in label ]
    loc = [ind for ind, el in enumerate(label_int) if (el in restrict_digits)]
    rest_image = img[loc, :]
    rest_label = label[loc, :] # Only the required indices
    rest_label = rest_label[:, restrict_digits] # only ther required columns
    return rest_image, rest_label

if restrict_digits is not None:
    train_image , train_label = restrict_data( train_image, train_label, restrict_digits )
    test_image , test_label = restrict_data( test_image, test_label, restrict_digits )
   

In [5]:
# Create a random ser of coordinates, and an adjacency matrix for it

def _random_coord(dims=[28,28], N=400):
    coord = [ [i,j] for i in range(dims[0]) for j in range(dims[1]) ]
    perm = np.random.permutation(coord)
    return perm[:N]

def _top_k(row, k=8):
    r = np.array(row)
    index = r.argsort()[-k]
    cutoff = r[index]
    return [0 if r<cutoff else r  for r in row]

def compute_A(k, n):
    index = _random_coord( N=n )
    N = len(index)
    A = []
    for i in range(N):
        row = [ np.linalg.norm(np.array(index[i]) - np.array(index[j]) ) for j in range(N)]
        A.append(_top_k(row, k))
    A = np.array(A)
    return index, A/(np.sqrt(2*27*27)/2)

# Given coordinates, extracts out the graph-signal from an image

def compute_f(I, index):
    I = I.reshape([28,28] )
    f=[]
    for i,j in index:
        f.append(I[i,j])
    return f


In [6]:
# Given images, returns the appropriately sampled graphs.
# Ns is the number of times each image is to be sampled, N is the number of nodes,  Nl is the number of labels
def image2graph(images, labels, k, Ns, Nl, N):
    f = np.zeros( [Ns*len(images) , N] )
    A = np.zeros( [Ns*len(images) , N, N] )
    l = np.zeros( [Ns*len(images), Nl] )
    for i in range(len(images)):
        print('Percentage complete : {0}'.format(i/len(images)*100))
        for n in range(Ns):
            coord, A[i*Ns + n, :, :] = compute_A(k, N)
            f[i*Ns + n, :] = compute_f( images[i,:] , coord)
            l[i*Ns + n, :] = labels[i,:]
    return f, A, l

In [7]:
train = image2graph( train_image, train_label, k, Ns, Nl, N)
test = image2graph( test_image, test_label, k, Ns, Nl, N)

Percentage complete : 0.0
Percentage complete : 0.008407600470825626
Percentage complete : 0.016815200941651252
Percentage complete : 0.025222801412476882
Percentage complete : 0.033630401883302505
Percentage complete : 0.042038002354128134
Percentage complete : 0.050445602824953764
Percentage complete : 0.05885320329577939
Percentage complete : 0.06726080376660501
Percentage complete : 0.07566840423743064
Percentage complete : 0.08407600470825627
Percentage complete : 0.09248360517908188
Percentage complete : 0.10089120564990753
Percentage complete : 0.10929880612073313
Percentage complete : 0.11770640659155877
Percentage complete : 0.1261140070623844
Percentage complete : 0.13452160753321002
Percentage complete : 0.14292920800403563
Percentage complete : 0.15133680847486128
Percentage complete : 0.1597444089456869
Percentage complete : 0.16815200941651254
Percentage complete : 0.17655960988733815
Percentage complete : 0.18496721035816377
Percentage complete : 0.1933748108289894
Perce

Percentage complete : 1.7151504960484276
Percentage complete : 1.7235580965192534
Percentage complete : 1.7319656969900792
Percentage complete : 1.7403732974609047
Percentage complete : 1.74878089793173
Percentage complete : 1.7571884984025559
Percentage complete : 1.7655960988733814
Percentage complete : 1.7740036993442072
Percentage complete : 1.782411299815033
Percentage complete : 1.7908189002858583
Percentage complete : 1.7992265007566839
Percentage complete : 1.8076341012275097
Percentage complete : 1.8160417016983355
Percentage complete : 1.824449302169161
Percentage complete : 1.8328569026399864
Percentage complete : 1.8412645031108121
Percentage complete : 1.849672103581638
Percentage complete : 1.8580797040524635
Percentage complete : 1.8664873045232888
Percentage complete : 1.8748949049941146
Percentage complete : 1.8833025054649404
Percentage complete : 1.891710105935766
Percentage complete : 1.9001177064065917
Percentage complete : 1.908525306877417
Percentage complete : 1

Percentage complete : 3.4555237935093324
Percentage complete : 3.4639313939801584
Percentage complete : 3.472338994450984
Percentage complete : 3.4807465949218095
Percentage complete : 3.4891541953926355
Percentage complete : 3.49756179586346
Percentage complete : 3.505969396334286
Percentage complete : 3.5143769968051117
Percentage complete : 3.5227845972759373
Percentage complete : 3.531192197746763
Percentage complete : 3.539599798217589
Percentage complete : 3.5480073986884144
Percentage complete : 3.55641499915924
Percentage complete : 3.564822599630066
Percentage complete : 3.573230200100891
Percentage complete : 3.5816378005717167
Percentage complete : 3.590045401042542
Percentage complete : 3.5984530015133678
Percentage complete : 3.606860601984194
Percentage complete : 3.6152682024550193
Percentage complete : 3.623675802925845
Percentage complete : 3.632083403396671
Percentage complete : 3.6404910038674965
Percentage complete : 3.648898604338322
Percentage complete : 3.6573062

Percentage complete : 5.221119892382714
Percentage complete : 5.229527492853539
Percentage complete : 5.237935093324365
Percentage complete : 5.246342693795191
Percentage complete : 5.2547502942660165
Percentage complete : 5.263157894736842
Percentage complete : 5.271565495207668
Percentage complete : 5.279973095678494
Percentage complete : 5.288380696149319
Percentage complete : 5.296788296620145
Percentage complete : 5.305195897090971
Percentage complete : 5.313603497561796
Percentage complete : 5.322011098032622
Percentage complete : 5.330418698503447
Percentage complete : 5.338826298974272
Percentage complete : 5.347233899445098
Percentage complete : 5.355641499915924
Percentage complete : 5.364049100386749
Percentage complete : 5.372456700857575
Percentage complete : 5.380864301328401
Percentage complete : 5.389271901799226
Percentage complete : 5.397679502270052
Percentage complete : 5.406087102740878
Percentage complete : 5.4144947032117035
Percentage complete : 5.42290230368252

Percentage complete : 6.986715991256095
Percentage complete : 6.99512359172692
Percentage complete : 7.003531192197747
Percentage complete : 7.011938792668572
Percentage complete : 7.020346393139398
Percentage complete : 7.0287539936102235
Percentage complete : 7.0371615940810495
Percentage complete : 7.045569194551875
Percentage complete : 7.0539767950227015
Percentage complete : 7.062384395493526
Percentage complete : 7.070791995964351
Percentage complete : 7.079199596435178
Percentage complete : 7.087607196906003
Percentage complete : 7.096014797376829
Percentage complete : 7.104422397847654
Percentage complete : 7.11282999831848
Percentage complete : 7.121237598789305
Percentage complete : 7.129645199260132
Percentage complete : 7.138052799730957
Percentage complete : 7.146460400201782
Percentage complete : 7.154868000672608
Percentage complete : 7.163275601143433
Percentage complete : 7.171683201614259
Percentage complete : 7.180090802085084
Percentage complete : 7.188498402555911

...

Percentage complete : 60.98012020342117
Percentage complete : 61.026352288488205
Percentage complete : 61.07258437355525
Percentage complete : 61.118816458622284
Percentage complete : 61.165048543689316
Percentage complete : 61.211280628756356
Percentage complete : 61.257512713823395
Percentage complete : 61.30374479889043
Percentage complete : 61.34997688395747
Percentage complete : 61.3962089690245
Percentage complete : 61.44244105409153
Percentage complete : 61.48867313915858
Percentage complete : 61.53490522422561
Percentage complete : 61.58113730929264
Percentage complete : 61.62736939435969
Percentage complete : 61.67360147942672
Percentage complete : 61.719833564493754
Percentage complete : 61.7660656495608
Percentage complete : 61.81229773462783
Percentage complete : 61.858529819694866
Percentage complete : 61.904761904761905
Percentage complete : 61.95099398982894
Percentage complete : 61.99722607489597
Percentage complete : 62.043458159963016
Percentage complete : 62.08969024

Percentage complete : 70.68885806749884
Percentage complete : 70.73509015256589
Percentage complete : 70.78132223763292
Percentage complete : 70.82755432269995
Percentage complete : 70.87378640776699
Percentage complete : 70.92001849283403
Percentage complete : 70.96625057790106
Percentage complete : 71.0124826629681
Percentage complete : 71.05871474803513
Percentage complete : 71.10494683310218
Percentage complete : 71.15117891816921
Percentage complete : 71.19741100323624
Percentage complete : 71.24364308830329
Percentage complete : 71.28987517337032
Percentage complete : 71.33610725843735
Percentage complete : 71.3823393435044
Percentage complete : 71.42857142857143
Percentage complete : 71.47480351363846
Percentage complete : 71.52103559870551
Percentage complete : 71.56726768377254
Percentage complete : 71.61349976883957
Percentage complete : 71.65973185390662
Percentage complete : 71.70596393897365
Percentage complete : 71.75219602404069
Percentage complete : 71.79842810910773
Pe

Percentage complete : 80.35136384650949
Percentage complete : 80.39759593157652
Percentage complete : 80.44382801664355
Percentage complete : 80.4900601017106
Percentage complete : 80.53629218677763
Percentage complete : 80.58252427184466
Percentage complete : 80.62875635691171
Percentage complete : 80.67498844197874
Percentage complete : 80.72122052704577
Percentage complete : 80.7674526121128
Percentage complete : 80.81368469717984
Percentage complete : 80.85991678224687
Percentage complete : 80.90614886731392
Percentage complete : 80.95238095238095
Percentage complete : 80.99861303744798
Percentage complete : 81.04484512251503
Percentage complete : 81.09107720758206
Percentage complete : 81.13730929264909
Percentage complete : 81.18354137771614
Percentage complete : 81.22977346278317
Percentage complete : 81.2760055478502
Percentage complete : 81.32223763291725
Percentage complete : 81.36846971798428
Percentage complete : 81.41470180305132
Percentage complete : 81.46093388811836
Per

Percentage complete : 90.01386962552012
Percentage complete : 90.06010171058715
Percentage complete : 90.10633379565418
Percentage complete : 90.15256588072123
Percentage complete : 90.19879796578826
Percentage complete : 90.24503005085529
Percentage complete : 90.29126213592234
Percentage complete : 90.33749422098937
Percentage complete : 90.3837263060564
Percentage complete : 90.42995839112345
Percentage complete : 90.47619047619048
Percentage complete : 90.52242256125751
Percentage complete : 90.56865464632455
Percentage complete : 90.61488673139159
Percentage complete : 90.66111881645863
Percentage complete : 90.70735090152566
Percentage complete : 90.75358298659269
Percentage complete : 90.79981507165972
Percentage complete : 90.84604715672677
Percentage complete : 90.8922792417938
Percentage complete : 90.93851132686083
Percentage complete : 90.98474341192788
Percentage complete : 91.03097549699491
Percentage complete : 91.07720758206194
Percentage complete : 91.12343966712899
Pe

Percentage complete : 99.67637540453075
Percentage complete : 99.72260748959778
Percentage complete : 99.76883957466481
Percentage complete : 99.81507165973186
Percentage complete : 99.86130374479889
Percentage complete : 99.90753582986592
Percentage complete : 99.95376791493297


In [8]:
filehandler = open(out_fname, 'wb')
pickle.dump([train, test], filehandler)

In [9]:
train_image.shape

(11894, 784)